<a href="https://colab.research.google.com/github/yonedahayato/stock_strategy/blob/feature%2F49/Numerai/my_Numerai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/yonedahayato/stock_strategy.git

Cloning into 'stock_strategy'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1911 (delta 82), reused 119 (delta 48), pack-reused 1744
Receiving objects: 100% (1911/1911), 1.94 MiB | 4.14 MiB/s, done.
Resolving deltas: 100% (1162/1162), done.


In [2]:
%cd stock_strategy/
!git checkout feature/49
!git fetch
!git rebase
%cd Numerai/

/content/stock_strategy
Branch 'feature/49' set up to track remote branch 'feature/49' from 'origin'.
Switched to a new branch 'feature/49'
First, rewinding head to replay your work on top of it...
Fast-forwarded feature/49 to refs/remotes/origin/feature/49.
/content/stock_strategy/Numerai


In [3]:
!pip install --upgrade pip
!pip install -r requirements.txt

     |████████████████████████████████| 1.5MB 7.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [1]:
from glob import glob
import numpy as np
import numerapi
import os
import pandas as pd

from mylogger import mylogger


In [5]:
from datasets import Features, print_log
class Datasets(object):
    """Datasets class

    データセットに関する処理をまとめる

    """
    def __init__(self, dir_="./dataset"):
        """__init__ func

        APIの準備
        パラメータの設置
        ディレクトリの作成

        Args:
            dir_ (str): データセットを保存するパス
        """
        self.napi = numerapi.NumerAPI(verbosity="info")
        self.dir_ = dir_

        os.makedirs(dir_, exist_ok=True)

    def print_log(self, message):
        print_log(message)

    def download(self):
        """download func

        numerapi を使ってデータをdownloadする

        """
        self.napi.download_current_dataset(unzip=True, dest_path=self.dir_)

    def load(self, test=False):
        """load func

        CSVデータをロードする

        Args:
            test (bool): テストデータかどうか

        Return:
            pandas.core.frame.DataFrame: 読み込んだデータ
        """
        files = os.listdir(path="{}/".format(self.dir_))
        dataset_names = [f for f in files if os.path.isdir(os.path.join(self.dir_, f))]
        sorted(dataset_names)
        base_path = os.path.join(self.dir_, "{}/numerai_{}_data.csv")
        if test:
            data_path = base_path.format(dataset_names[0], "tournament")
        else:
            data_path = base_path.format(dataset_names[0], "training")

        self.print_log(data_path)
        self.row_data = pd.read_csv(data_path)

        return self.row_data

    def static(self):
        """static func

        データの統計情報を表示する

        """
        self.print_log("row: {}, colmun: {}".format(self.row_data.shape[0], self.row_data.shape[1]))
        self.print_log(self.row_data.head())

        self.features = Features(self.row_data.columns)
 
        # 平均の計算
        mean_df = self.row_data.mean()
        mean_df.plot(legend=False, kind='hist', title="各列の平均のヒストグラム")


In [2]:
from datasets import Datasets
datasets = Datasets()
datasets.download()
data_df = datasets.load()

./dataset/numerai_dataset_221.zip: 100%|█████████▉| 382M/382M [00:24<00:00, 14.1MB/s] 2020-07-19 08:17:05,040 INFO numerapi.base_api: unzipping file...
[I 200719 08:17:19 datasets:16] ./dataset/numerai_dataset_221/numerai_training_data.csv
./dataset/numerai_dataset_221.zip: 382MB [00:40, 14.1MB/s]                           

In [7]:
# datasets.static()

In [3]:
!pip install xgboost

     |████████████████████████████████| 127.6 MB 23 kB/s /s eta 0:00:01


In [6]:
from xgboost import XGBRegressor

feature_names = [f for f in datasets.training_data.columns if "feature" in f]
model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1)
model.fit(datasets.training_data[feature_names], datasets.training_data["target_kazutsugi"])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [7]:
test_data = datasets.load(test=True)

[I 200719 09:24:40 datasets:16] ./dataset/numerai_dataset_221/numerai_tournament_data.csv


In [8]:
predictions = model.predict(test_data[feature_names])

In [27]:
predictions = pd.DataFrame(predictions)
ranked_prediction = predictions.rank(pct=True, method="first")
test_data["target_kazutsugi"] = test_data["target_kazutsugi"].astype('float16')

float16


In [36]:
ranked_prediction[0] = ranked_prediction[0].astype('float16')

float16


In [40]:
print(test_data["target_kazutsugi"].shape)
print(ranked_prediction[0].shape)

# correlation = np.corrcoef(test_data[["target_kazutsugi"]], ranked_prediction)[0, 1]
correlation = test_data["target_kazutsugi"].corr(ranked_prediction[0])
print(correlation)

(1596622,)
(1596622,)
0.028008976753021792
